<a href="https://colab.research.google.com/github/tobyloby12/AMLS-assignment/blob/main/transfer_learning_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
# load model without output layer
model = VGG16(include_top=False, input_shape=(224, 224, 3))

 2277376/58889256 [>.............................] - ETA: 1:14:32

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(4, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# trainable layers here
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = True

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
import cv2
# importing the training data

# loading in images and turing it into an array
path = '/content/gdrive/MyDrive/amlsAssignment'

path = os.path.join(path, 'dataset')

df_labels = pd.read_csv(os.path.join(path, 'label.csv'))

# getting train and test dataframes
train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

classes = {'meningioma_tumor': 0, 'no_tumor': 1, 'glioma_tumor': 2, 'pituitary_tumor': 3}

def feature_extraction(dataset, img_path, name):
    images = []
    labels = []
    for row in tqdm(dataset.iterrows(), total=dataset.shape[0], desc="Progress"):
        image_name = row[1][0]
        label = row[1][1]
        # opening and flattening image
        img = cv2.imread(os.path.join(img_path, image_name))
        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
        labels.append(classes[label])
        images.append(img)

    return images, labels

train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')

x_train, y_train = feature_extraction(train, train_path, 'train')
x_test, y_test = feature_extraction(test, test_path, 'test')

print('Train size: ', len(x_train))
print('Test size: ', len(x_test))

Progress:   0%|          | 0/2100 [00:00<?, ?it/s]

Progress:   0%|          | 0/900 [00:00<?, ?it/s]

Train size:  2100
Test size:  900


In [ ]:
import tensorflow as tf
import numpy as np
y_train = tf.one_hot(y_train, depth=len(classes))

In [ ]:
np.save('/content/gdrive/MyDrive/amlsAssignment/dataset/x_train', x_train)
np.save('/content/gdrive/MyDrive/amlsAssignment/dataset/y_train', y_train)
np.save('/content/gdrive/MyDrive/amlsAssignment/dataset/x_test', x_test)
np.save('/content/gdrive/MyDrive/amlsAssignment/dataset/y_test', y_test)

In [ ]:
import tensorflow as tf
import numpy as np
# compiling the model and training
x_train = np.load('/content/gdrive/MyDrive/amlsAssignment/dataset/x_train')
y_train = np.load('/content/gdrive/MyDrive/amlsAssignment/dataset/y_train')
x_test = np.load('/content/gdrive/MyDrive/amlsAssignment/dataset/x_test')
y_test = np.load('/content/gdrive/MyDrive/amlsAssignment/dataset/y_test')



model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.TruePositives()])

history = model.fit(
    x=np.array(x_train), y=np.array(y_train), batch_size=32, epochs=50
)

Epoch 1/50
 1/66 [..............................] - ETA: 1:06:42 - loss: 8.3918 - binary_accuracy: 0.6250 - true_positives: 8.0000

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/gdrive/MyDrive/amlsAssignment/model')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

y = list(range(1, 101))
plt.scatter(y, history.history['accuracy'])